# A pipeline for processing & successfully merging ephys and tracking data for further analyses.
### author: github/bartulem


#### **[Step 1]** Merge Neuropixel sessions to run kilosort. This step is *optional*, you can skip it if you are interested in only one session.
In the cell below, you determine the directories where the Neuropixel .bin files are (they should all be in one directory!), the desired name of the future merged file and its destination, the number of channels on the probe and whether to do the merging through the cmd prompt. This code outputs a .pkl file with information about changepoints of the merged sessions.

In [ ]:
import concatenateNPX

fileDirectories = [r'D:\SGL_DATA\test_concatenation']
newFileNames = [r'D:\SGL_DATA\test_concatenation\05022020_both_distal_g0_t0.imec0.ap.bin']
cmdPrompt = 1
nchan = 385

In [ ]:
for fileDir, newFileName in zip(fileDirectories, newFileNames): 
    concatClass = concatenateNPX.concat(fileDir, newFileName)
    concatClass.concatNPX(cmdPrompt=cmdPrompt, nchan=nchan)

#### **[Step 2]** Run kilosort through Python.
This assumes you are happy with *everything* in the config file. If you need to mess with that, either the code needs to change or you do this step in Matlab.
If this doesn't bother you, then you should do the following:
1. Install matlab.engine: https://www.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html (as an admin!).
2. Kilosort runs on all the .bin files in the given directory below. Make sure that this is what you want.
2. Run the cell below.

In [ ]:
import runKilosort

fileDIR = r'D:\SGL_DATA\test_concatenation'
kilosortDIR = r'A:\group\bartulm\Kilosort2-master'

In [ ]:
runKilosort.runKilo(fileDIR, kilosortDIR)

#### **[Step 3]** Choose good clusters in Phy.
1. Install Phy: https://github.com/cortex-lab/phy
2. Navigate to the directory where kilosort was run, open powershell and type "cmd", followed by "activate phy2", followed by "phy template-gui params.py".
3. Complete the manual curation (tutorial here: https://phy.readthedocs.io/en/latest/) and save your work.

#### **[Step 4]** Split clusters back into individual sessions and get spike times. This step should be completed also if you only spike sorted one session.
Note that you set the number of channels on the probe, whether you have only one session or not, and the minimum number of spikes in one session to consider the cluster worthy of saving.

In [ ]:
import saveSpikeTimes

thedir = r'A:\store\Bartul\neuropixel\distal05022020'
pklfile = '05022020_both_distal_g0_t0.imec0.ap.pkl'
nchan = 385
onesession = 1
minspikes = 100

In [ ]:
sstClass = saveSpikeTimes.sst(thedir)
sstClass.splitClusters(onesession=onesession, minspikes=minspikes, nchan=nchan, pklfile=pklfile)

#### **[Step 5]** Read in the sync events (make sure the PC has enough memory to run this, say 64Gb RAM) and put them in a separate .txt file.
You set the list with all the files whose sync events you'd like to read, and likewise the paths of future .txt files with that content. There's also the channel number parameter, which means the same as above.

In [ ]:
import readSyncEvents

npxList = [r'D:\SGL_DATA\05022020_session2_distal_g0\05022020_session2_distal_g0_t0.imec0.ap.bin']
npxTXTs = [r'A:\store\Bartul\neuropixel\05022020_distal_session2.txt']
nchan = 385

In [ ]:
for npxFile, txtFile in zip(npxList, npxTXTs): 
    readClass = readSyncEvents.read(npxFile, txtFile)
    readClass.readSE(nchan=nchan)

## The next 4 steps are optional, and they should be completed if you want to know whether the systems are synced properly (Neuropixel, OptiTrack and Teensy). Otherwise jump to step 10.


#### **[Step 6]** Load the sync data from the .txt file(s) and estimate the frames of LEDon appearances in Motive to facilitate the search.
You set the dirs where the .txt files are.

In [ ]:
import estimateMotiveFrames

txtdirs = [r'A:\store\Bartul\neuropixel']

In [ ]:
emfClass = estimateMotiveFrames.emf(txtdirs)
syncData = emfClass.estimateMF()

#### **[Step 7]** Go to Motive and find the true LEDon appearances and place them into the appropriate dataframes.

In [ ]:
syncData[r'A:\store\Bartul\neuropixel']['05022020_distal_session2'].loc[:, 'Opti (true frame)'] = [0, 242, 13894, 23295, 33193, 40445, 62561, 67635, 67712, 73843, 74128]

#### **[Step 8]** Do the regression to calculate 'Npx pred (sec)' and see how well it does.
This outputs a .csv file with all the details of the computations and the empirical frame rate (first row, last column).

In [ ]:
import syncRegression

lrClass = syncRegression.lr(syncData)
syncData = lrClass.linreg()

#### **[Step 9]** Check the IMU sync with Npx and save the IMU file as a pickled dataframe.

In [ ]:
import syncIMU

txtsIMU = [r'A:\store\Bartul\neuropixel\scripts\CoolTerm Capture 2020-02-12 14-43-06.txt']
txtsNPX = [r'A:\store\Bartul\neuropixel\scripts\12022020_distal_session1.txt']

In [ ]:
imuSClass = syncIMU.imuS(txtsIMU, txtsNPX)
imuSClass.syncMilis()

## The optional check steps stop here.

#### **[Step 10]** Create .pkl file for GUI.
1. If you haven't done so yet, label the rigid body and the body marker set in Motive (read the tutorial if you need).
2. File > Export Tracking Data
3. ***[important]*** Start Frame > Custom > 1st LED light frame, End Frame > Custom > Last LED light frame
4. Put OFF in the following variables: (1) Unlabeled markers, (2) Rigid Bodies, (3) Rigid Body markers, (4) Bones, (5) Bone markers
5. Click "Export" and you should create a .csv file
6. Run the cell below.

In [ ]:
import motive2GUI

thecsvs = [r'A:\store\Bartul\neuropixel\distal05022020\tracking\Take 2020-02-05 04.01.13 PM (2).csv']
thetxts = [r'A:\store\Bartul\neuropixel\05022020_distal_session2.txt']
framerates = [r'A:\store\Bartul\neuropixel\05022020_distal_session2.csv']

In [ ]:
for thecsv, thetxt, framerate in zip(thecsvs, thetxts, framerates): 
    mtgClass = motive2GUI.mtg(thecsv, thetxt)
    mtgClass.csvTOpkl(framerate=framerate)

#### **[Step 11]** Create the head in the GUI, load the spiking .mat files and export everything as a .mat file. Now you are ready to analyze!